In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import pytorch_lightning as pl
import os
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
import pandas as pd
from sklearn.model_selection import train_test_split
from torchmetrics.functional import accuracy
from sklearn import metrics
import torch
import csv
from sklearn import metrics
from matplotlib import pyplot as plt
import math
import copy
import random
import sys

torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)


tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import importlib

# import outsiders
sys.path.append('../pre_processing_and_recover_network')
import data_preprocessing
importlib.reload(data_preprocessing)

sys.path.append('../pytorch_model')
import ECG
importlib.reload(ECG)


In [2]:
swell_data = data_preprocessing.load_data(os.path.join("../pre_processing_and_recover_network/swell_dataset/","swell_dict.npy"))
swell_data = data_preprocessing.swell_prepare_for_10fold(swell_data)
before=copy.copy(swell_data)
#fix the range of the labels to init from value of zero
swell_data[:,1:4]-=1

**For spliting subjests for Train and Test**

In [3]:
def train_test_indexes_var2(ratio, objects):
    # Split objects into train and test sets
    num_test = int(len(set(objects)))-int(len(set(objects)) * ratio)
    obj_list = list(set(objects))
    random.shuffle(obj_list)
    test_objs = obj_list[:num_test]
    train_objs = obj_list[num_test:]

    train_idxs = []
    test_idxs = []

    # Assign each object's samples to the appropriate set
    for obj in train_objs:
        obj_idxs = [i for i, o in enumerate(objects) if o == obj]
        train_idxs.extend(obj_idxs)

    for obj in test_objs:
        obj_idxs = [i for i, o in enumerate(objects) if o == obj]
        test_idxs.extend(obj_idxs)

    # Shuffle the final train and test sets
    random.shuffle(train_idxs)
    random.shuffle(test_idxs)

    return (train_idxs), (test_idxs)
# check train_test_indexes_var2()
# not for each runnig acieve exaclty 9:1 train:test
ratio = 0.9
objects=swell_data[:,0]
print("tot len is ", len(objects))
print("trin len is ",ratio*len(objects))
print("test len is ",len(objects)-ratio*len(objects))
i=train_test_indexes_var2(ratio, objects)


print("train obj:")
tr=[]
for j in i[0]:
  tr.append(objects[int(j)])
tr_u=set(tr)

print("test obj:")
te=[]
for j in i[1]:
  te.append(objects[int(j)])
te_u=set(te)

print(tr_u,len(tr))
print(te_u,len(te))
splits=i
np.save("./splits_B",splits)

tot len is  14595
trin len is  13135.5
test len is  1459.5
train obj:
test obj:
{2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 9.0, 10.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0} 12567
{1.0, 18.0, 25.0} 2028


/home1/eitan.c@staff.technion.ac.il/ECG_proj/dockvenv/code/venv/lib/python3.8/site-packages/numpy/lib/npyio.py:521: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [4]:
x_tr=[]
y_tr=[]
for i in splits[0]:
    x_tr.append(swell_data[i,4:])
    y_tr.append(swell_data[i,:4])
x_train=np.array(x_tr)
y_train=np.array(y_tr)

x_te=[]
y_te=[]
for i in splits[1]:
    x_te.append(swell_data[i,4:])
    y_te.append(swell_data[i,:4])
x_test=np.array(x_te)
y_test=np.array(y_te)

In [16]:
(x_train).shape
print(type(x_train))

<class 'numpy.ndarray'>


In [5]:
x_train = torch.from_numpy(x_train).float().unsqueeze(1)
y_train = torch.tensor(y_train).long()
x_test = torch.from_numpy(x_test).float().unsqueeze(1)
y_test = torch.tensor(y_test).long()
tensor_dataset = torch.utils.data.TensorDataset(x_train, y_train)
tensor_dataset_test = torch.utils.data.TensorDataset(x_test, y_test)
from sklearn import metrics

train_loader = DataLoader(tensor_dataset, batch_size=128) # for debug set num_workers=0
test_loader = DataLoader(tensor_dataset_test,batch_size=128) # for debug set num_workers=0

In [ ]:
# model
model =ECG.EmotionRec()
# training
trainer = pl.Trainer(gpus = 1 , max_epochs = 100)
torch.save(model.state_dict(), './model_light_100_B.pt')
trainer.fit(model, train_loader,test_loader)

trainer = pl.Trainer(gpus = 1 , max_epochs = 150)
torch.save(model.state_dict(), './model_light_250_B.pt')
trainer.fit(model, train_loader,test_loader)

**For randomal splits**

In [ ]:
def split_data_indexes(len_data, train_ratio):
    indexes = list(range(len_data))
    random.shuffle(indexes)
    train_size = int(len_data * train_ratio)
    train_indexes = indexes[:train_size]
    test_indexes = indexes[train_size:]
    return (train_indexes, test_indexes)

splits=split_data_indexes(swell_data.shape[0],0.9)
np.save("./splits_A",splits)
x_tr=[]
y_tr=[]
for i in splits[0]:
    x_tr.append(swell_data[i,4:])
    y_tr.append(swell_data[i,:4])
x_train=np.array(x_tr)
y_train=np.array(y_tr)

x_te=[]
y_te=[]
for i in splits[1]:
    x_te.append(swell_data[i,4:])
    y_te.append(swell_data[i,:4])
x_test=np.array(x_te)
y_test=np.array(y_te)

(x_train).shape
print(type(x_train))

x_train = torch.from_numpy(x_train).float().unsqueeze(1)
y_train = torch.tensor(y_train).long()
x_test = torch.from_numpy(x_test).float().unsqueeze(1)
y_test = torch.tensor(y_test).long()
tensor_dataset = torch.utils.data.TensorDataset(x_train, y_train)
tensor_dataset_test = torch.utils.data.TensorDataset(x_test, y_test)
from sklearn import metrics

train_loader = DataLoader(tensor_dataset, batch_size=128) # for debug set num_workers=0
test_loader = DataLoader(tensor_dataset_test,batch_size=128) # for debug set num_workers=0

In [ ]:
# model
model =ECG.EmotionRec()
# training
trainer = pl.Trainer(gpus = 1 , max_epochs = 100)
torch.save(model.state_dict(), './model_light_100.pt')
trainer.fit(model, train_loader,test_loader)

trainer = pl.Trainer(gpus = 1 , max_epochs = 150)
torch.save(model.state_dict(), './model_light_250.pt')
trainer.fit(model, train_loader,test_loader)